In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from statsmodels.tsa.filters.hp_filter import hpfilter
import matplotlib.pyplot as plt

In [ ]:
try:
    # 尝试使用GBK编码读取文件
    china_data = pd.read_csv('china_economic_data.csv', encoding='GBK')
except UnicodeDecodeError:
    try:
        # 若GBK编码失败，尝试使用GB2312编码
        china_data = pd.read_csv('china_economic_data.csv', encoding='GB2312')
    except UnicodeDecodeError:
        print("无法确定文件编码格式，请手动检查文件编码。")
        exit(1)

In [ ]:
# 重新设置列名
china_data.columns = ['Indicator', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015',
                      '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006',
                      '2005', '2004', '2003', '2002', '2001', '2000', '1999', '1998', '1997',
                      '1996', '1995', '1994', '1993', '1992', '1991', '1990', '1989', '1988',
                      '1987', '1986', '1985', '1984', '1983', '1982', '1981', '1980', '1979',
                      '1978']

In [ ]:
# 从第2行(index = 1)开始加载数据
china_data = china_data[1:]

In [ ]:
# 重置索引
china_data = china_data.reset_index(drop=True)

In [ ]:
# 将除了`指标`列以外的列转换为数值类型
for col in china_data.columns[1:]:
    china_data[col] = pd.to_numeric(china_data[col], errors='coerce')

In [ ]:
# 处理可能存在的空值数据，删除含有过多空值的行或列（可根据实际情况调整）
china_data = china_data.dropna(how='all')
china_data = china_data.dropna(axis=1, thresh=len(china_data) * 0.5)

In [ ]:
# 转置数据，使得指标为列名，年份为索引
china_data = china_data.set_index('Indicator').T

In [ ]:
# 计算每个指标的描述性统计
stats = {}
for col in china_data.columns:
    values = china_data[col].dropna()
    if len(values) > 0:
        mean = values.mean()
        median = values.median()
        std_dev = values.std()
        min_val = values.min()
        max_val = values.max()
        skewness = skew(values)
        kurt_value = kurtosis(values)
        stats[col] = {
            'Mean': mean,
            'Median': median,
            'Standard Deviation': std_dev,
            'Minimum': min_val,
            'Maximum': max_val,
            'Skewness': skewness,
            'Kurtosis': kurt_value
        }
    else:
        stats[col] = {
            'Mean': np.nan,
            'Median': np.nan,
            'Standard Deviation': np.nan,
            'Minimum': np.nan,
            'Maximum': np.nan,
            'Skewness': np.nan,
            'Kurtosis': np.nan
        }

In [ ]:
# 输出每个指标的描述性统计
for col, stat in stats.items():
    print(f"Descriptive statistics for indicator {col}:")
    for key, value in stat.items():
        print(f"{key}: {value}")
    print()

In [ ]:
# 提取中国实际GDP数据
real_gdp = china_data['国内实际生产总值（亿元）']
real_gdp.index = pd.to_datetime(real_gdp.index.str.replace('年', ''), format='%Y')
real_gdp = real_gdp.sort_index()

In [ ]:
# 计算中国实际GDP的10年滚动标准差
rolling_std = real_gdp.rolling(window=10).std()
print("10-year rolling standard deviation of China's real GDP:")
print(rolling_std)

In [ ]:
# 设置图片中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 绘制中国实际GDP的10年滚动标准差图
plt.figure(figsize=(12, 6))
plt.plot(rolling_std.index, rolling_std.values, label='10-Year Rolling Standard Deviation of China Real GDP')
plt.title('10-Year Rolling Standard Deviation of China Real GDP')
plt.xlabel('Year')
plt.xticks(rotation=45)
plt.ylabel('Standard Deviation')
plt.legend()
plt.show()

In [ ]:
# 读取美国GDP数据
try:
    usa_data = pd.read_excel('usa数据（经过转换后）工作簿1.xlsx', engine='openpyxl')
    if usa_data.shape[1] == 2:
        usa_data.columns = ['year', 'RGDP_CNY']
        usa_data['year'] = pd.to_datetime(usa_data['year'], format='%Y')
        usa_data = usa_data.set_index('year')
    else:
        print(f"The number of columns in the US data is {usa_data.shape[1]}, which does not match the expected 2 columns. Please check the data file.")
        exit(1)
except FileNotFoundError:
    print("The file 'usa数据（经过转换后）工作簿1.xlsx' was not found. Please check the file path.")
    exit(1)
except Exception as e:
    print(f"An error occurred while reading the US GDP data: {e}")
    exit(1)

In [ ]:
# 确保中美数据时间范围一致
common_index = real_gdp.index.intersection(usa_data.index)
real_gdp = real_gdp[common_index]
usa_gdp = usa_data.loc[common_index, 'RGDP_CNY']

In [ ]:
# 确保中美数据没有缺失值，否则可能影响Hodrick - Prescott滤波
real_gdp = real_gdp.dropna()
usa_gdp = usa_gdp.dropna()

In [ ]:
# 应用Hodrick - Prescott滤波器将美国和中国的GDP分解为趋势和周期性成分
lambda_value = 1600
china_cycle, china_trend = hpfilter(real_gdp, lamb=lambda_value)
usa_cycle, usa_trend = hpfilter(usa_gdp, lamb=lambda_value)

In [ ]:
# 绘制中美GDP周期成分图
plt.figure(figsize=(12, 6))
plt.plot(china_cycle.index, china_cycle.values, label="China's GDP Cyclical Component")
plt.plot(usa_cycle.index, usa_cycle.values, label='US GDP Cyclical Component')
plt.xlabel('Year')
plt.xticks(rotation=45)
plt.ylabel('Cyclical Component')
plt.title('Cyclical Components of China and US GDP')
plt.legend()
plt.show()

In [ ]:
# 计算中美GDP周期成分的相关系数
correlation = china_cycle.corr(usa_gdp)
print(f"Overall correlation coefficient of the cyclical components of China and US GDP: {correlation}")

In [ ]:
# 计算2000年、2008年、2015年前后的相关系数
years = [2000, 2008, 2015]
correlation_stats = {}
for year in years:
    before_china = china_cycle[china_cycle.index < f'{year}-01-01']
    before_usa = usa_gdp[usa_gdp.index < f'{year}-01-01']
    after_china = china_cycle[china_cycle.index >= f'{year}-01-01']
    after_usa = usa_gdp[usa_gdp.index >= f'{year}-01-01']

    if len(before_china) > 0 and len(before_usa) > 0:
        correlation_before = before_china.corr(before_usa)
        print(f"Correlation coefficient of the cyclical components of China and US GDP before {year}: {correlation_before}")
        correlation_stats[f'Before {year}'] = correlation_before
    else:
        print(f"Insufficient data before {year} to calculate the correlation coefficient.")
        correlation_stats[f'Before {year}'] = np.nan

    if len(after_china) > 0 and len(after_usa) > 0:
        correlation_after = after_china.corr(after_usa)
        print(f"Correlation coefficient of the cyclical components of China and US GDP after {year}: {correlation_after}")
        correlation_stats[f'After {year}'] = correlation_after
    else:
        print(f"Insufficient data after {year} to calculate the correlation coefficient.")
        correlation_stats[f'After {year}'] = np.nan
    print()

In [ ]:
# 将描述性统计结果转换为DataFrame
desc_stats_df = pd.DataFrame(stats).T

In [ ]:
# 将中美周期成分转换为DataFrame
cycle_df = pd.DataFrame({
    'China GDP Cyclical Component': china_cycle,
    'US GDP Cyclical Component': usa_cycle
})

In [ ]:
# 将相关系数结果转换为DataFrame
correlation_df = pd.DataFrame.from_dict(correlation_stats, orient='index', columns=['Correlation Coefficient'])
correlation_df.loc['Overall Correlation Coefficient'] = correlation

In [ ]:
# 导出到Excel文件
with pd.ExcelWriter('4economic_stats.xlsx') as writer:
    desc_stats_df.to_excel(writer, sheet_name='Descriptive Statistics')
    cycle_df.to_excel(writer, sheet_name='China-US Cyclical Components')
    correlation_df.to_excel(writer, sheet_name='Correlation Coefficients')
    rolling_std.to_excel(writer, sheet_name='10-Year Rolling Standard Deviation of China Real GDP')